In [1]:
from __future__ import print_function
import unicodedata
import re
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense,CuDNNLSTM
import numpy as np

In [2]:
def unicode_2_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    s = unicode_2_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [29]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'por-eng/por.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    input_text = normalize_string(input_text) 
    target_text = normalize_string(target_text)
    target_text ='\t' +  target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])


encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char.lower()]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


Number of samples: 10000
Number of unique input tokens: 30
Number of unique output tokens: 32
Max sequence length for inputs: 18
Max sequence length for outputs: 45


In [30]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = CuDNNLSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 3s 324us/step - loss: 0.9919 - val_loss: 1.0054
Epoch 2/100
8000/8000 [==============================] - 2s 225us/step - loss: 0.7674 - val_loss: 0.8539
Epoch 3/100
8000/8000 [==============================] - 2s 232us/step - loss: 0.6734 - val_loss: 0.7782
Epoch 4/100
8000/8000 [==============================] - 2s 240us/step - loss: 0.6214 - val_loss: 0.7216
Epoch 5/100
8000/8000 [==============================] - 2s 231us/step - loss: 0.5796 - val_loss: 0.6858
Epoch 6/100
8000/8000 [==============================] - 2s 226us/step - loss: 0.5422 - val_loss: 0.6516
Epoch 7/100
8000/8000 [==============================] - 2s 227us/step - loss: 0.5113 - val_loss: 0.6219
Epoch 8/100
8000/8000 [==============================] - 2s 225us/step - loss: 0.4851 - val_loss: 0.5958
Epoch 9/100
8000/8000 [==============================] - 2s 229us/step - loss: 0.4615 - val_loss: 0.5823
Epoch 1

Epoch 78/100
8000/8000 [==============================] - 2s 228us/step - loss: 0.0716 - val_loss: 0.7555
Epoch 79/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.0701 - val_loss: 0.7578
Epoch 80/100
8000/8000 [==============================] - 2s 229us/step - loss: 0.0690 - val_loss: 0.7666
Epoch 81/100
8000/8000 [==============================] - 2s 228us/step - loss: 0.0676 - val_loss: 0.7727
Epoch 82/100
8000/8000 [==============================] - 2s 225us/step - loss: 0.0665 - val_loss: 0.7755
Epoch 83/100
8000/8000 [==============================] - 2s 226us/step - loss: 0.0652 - val_loss: 0.7777
Epoch 84/100
8000/8000 [==============================] - 2s 230us/step - loss: 0.0649 - val_loss: 0.7801
Epoch 85/100
8000/8000 [==============================] - 2s 238us/step - loss: 0.0631 - val_loss: 0.7889
Epoch 86/100
8000/8000 [==============================] - 2s 222us/step - loss: 0.0620 - val_loss: 0.7968
Epoch 87/100
8000/8000 [======================

In [31]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [49]:
for seq_index in range(500):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: go .
Decoded sentence: va .

-
Input sentence: go .
Decoded sentence: va .

-
Input sentence: hi .
Decoded sentence: oi .

-
Input sentence: run !
Decoded sentence: corra !

-
Input sentence: run !
Decoded sentence: corra !

-
Input sentence: run !
Decoded sentence: corra !

-
Input sentence: run .
Decoded sentence: corra !

-
Input sentence: run .
Decoded sentence: corra !

-
Input sentence: run .
Decoded sentence: corra !

-
Input sentence: who ?
Decoded sentence: quem ?

-
Input sentence: wow !
Decoded sentence: nossa !

-
Input sentence: wow !
Decoded sentence: nossa !

-
Input sentence: wow !
Decoded sentence: nossa !

-
Input sentence: fire !
Decoded sentence: fogo !

-
Input sentence: help !
Decoded sentence: ajuda !

-
Input sentence: help !
Decoded sentence: ajuda !

-
Input sentence: jump !
Decoded sentence: pule !

-
Input sentence: jump .
Decoded sentence: pulem .

-
Input sentence: jump .
Decoded sentence: pulem .

-
Input sentence: stop !
Decoded sentenc

-
Input sentence: goodbye !
Decoded sentence: ate mais ver !

-
Input sentence: goodbye !
Decoded sentence: ate mais ver !

-
Input sentence: goodbye !
Decoded sentence: ate mais ver !

-
Input sentence: goodbye !
Decoded sentence: ate mais ver !

-
Input sentence: goodbye !
Decoded sentence: ate mais ver !

-
Input sentence: hang on !
Decoded sentence: aguardem .

-
Input sentence: hang on !
Decoded sentence: aguardem .

-
Input sentence: hang on .
Decoded sentence: aguardem .

-
Input sentence: hang on .
Decoded sentence: aguardem .

-
Input sentence: he came .
Decoded sentence: ele veio .

-
Input sentence: he runs .
Decoded sentence: ele corre .

-
Input sentence: help me !
Decoded sentence: ajudem me !

-
Input sentence: help me !
Decoded sentence: ajudem me !

-
Input sentence: help me !
Decoded sentence: ajudem me !

-
Input sentence: help me .
Decoded sentence: ajudem me .

-
Input sentence: help me .
Decoded sentence: ajudem me .

-
Input sentence: help me .
Decoded sentence: 

-
Input sentence: call tom .
Decoded sentence: ligue para o tom .

-
Input sentence: cheer up !
Decoded sentence: animo !

-
Input sentence: cool off !
Decoded sentence: acalme se !

-
Input sentence: cool off !
Decoded sentence: acalme se !

-
Input sentence: cuff him .
Decoded sentence: algeme o .

-
Input sentence: don t go .
Decoded sentence: nao va .

-
Input sentence: drive on .
Decoded sentence: dirija !

-
Input sentence: find tom .
Decoded sentence: encontrem o tom .

-
Input sentence: find tom .
Decoded sentence: encontrem o tom .

-
Input sentence: fix this .
Decoded sentence: conserta isto .

-
Input sentence: get away !
Decoded sentence: fora !

-
Input sentence: get away !
Decoded sentence: fora !

-
Input sentence: get away !
Decoded sentence: fora !

-
Input sentence: get down !
Decoded sentence: desca !

-
Input sentence: get down !
Decoded sentence: desca !

-
Input sentence: get down .
Decoded sentence: escreva .

-
Input sentence: get down .
Decoded sentence: escrev

-
Input sentence: it s red .
Decoded sentence: e vermelha .

-
Input sentence: it s red .
Decoded sentence: e vermelha .

-
Input sentence: it s red .
Decoded sentence: e vermelha .

-
Input sentence: it s sad .
Decoded sentence: e triste .

-
Input sentence: keep out !
Decoded sentence: nao entre .

-
Input sentence: keep out !
Decoded sentence: nao entre .

-
Input sentence: keep out .
Decoded sentence: nao entre .

-
Input sentence: keep out .
Decoded sentence: nao entre .

-
Input sentence: kiss tom .
Decoded sentence: beije tom .

-
Input sentence: leave it .
Decoded sentence: deixe para la .

-
Input sentence: leave me .
Decoded sentence: me deixe .

-
Input sentence: leave me .
Decoded sentence: me deixe .

-
Input sentence: leave me .
Decoded sentence: me deixe .

-
Input sentence: leave us .
Decoded sentence: nos deixe .

-
Input sentence: leave us .
Decoded sentence: nos deixe .

-
Input sentence: let s go !
Decoded sentence: vamos !

-
Input sentence: let s go .
Decoded sent

In [52]:
input_text_encode = np.zeros((1, max_encoder_seq_length, num_encoder_tokens))

input_seq = 'who went'

for i, char in enumerate(input_seq):
    input_text_encode[0, i, input_token_index[char]] = 1
    
print('Tradução: ', decode_sequence(input_text_encode))

Tradução:  quem encontrou ?



In [ ]:
target_texts

In [ ]:
model.predict()